# Leitura de dados SQL com Python

## Usando `SQLite3`

Bibliotecas importadas nesta seção:

In [1]:
# Bibliotecas importadas
import os
import sqlite3

In [2]:
# Diretório fonte de dados
data_dirpath = r"../../data"

Conexão com o banco de dados usando a biblioteca `sqlite3`:

In [3]:
connection = sqlite3.connect(os.path.join(data_dirpath, 'chinook.db'))
cursor = connection.cursor()

O código seguinte executa uma consulta SQL e lista as tabelas existentes no banco de dados "chinook.db":

In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
cursor.fetchall()

[('albums',),
 ('sqlite_sequence',),
 ('artists',),
 ('customers',),
 ('employees',),
 ('genres',),
 ('invoices',),
 ('invoice_items',),
 ('media_types',),
 ('playlists',),
 ('playlist_track',),
 ('tracks',),
 ('sqlite_stat1',)]

Nas análises seguintes, são considerados os dados referentes às faturas, constantes nas tabelas _invoices_ e *invoice_items*.

A tabela de faturas (_invoices_) contêm os seguintes campos:

In [5]:
cursor.execute('PRAGMA table_info(invoices)')
cursor.fetchall()

[(0, 'InvoiceId', 'INTEGER', 1, None, 1),
 (1, 'CustomerId', 'INTEGER', 1, None, 0),
 (2, 'InvoiceDate', 'DATETIME', 1, None, 0),
 (3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0),
 (4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0),
 (5, 'BillingState', 'NVARCHAR(40)', 0, None, 0),
 (6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0),
 (7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0),
 (8, 'Total', 'NUMERIC(10,2)', 1, None, 0)]

Por sua vez, a tabela de items de fatura (*invoice_items*) contém os campos:

In [6]:
cursor.execute('PRAGMA table_info(invoice_items)')
cursor.fetchall()

[(0, 'InvoiceLineId', 'INTEGER', 1, None, 1),
 (1, 'InvoiceId', 'INTEGER', 1, None, 0),
 (2, 'TrackId', 'INTEGER', 1, None, 0),
 (3, 'UnitPrice', 'NUMERIC(10,2)', 1, None, 0),
 (4, 'Quantity', 'INTEGER', 1, None, 0)]

Consulta aos primeiros 05 (cinco) registros da tabela **artists** do banco de dados **chinook.db**.

In [7]:
cursor.execute('SELECT * FROM artists LIMIT 5;')
cursor.fetchall()

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains')]

Consulta das 05 (cinco) maiores _invoices_ ordenadas pelo respectivo montante:

In [8]:
query = """
    SELECT InvoiceDate, Total 
    FROM invoices
    ORDER BY Total DESC
    LIMIT 5;
"""

cursor.execute(query)
cursor.fetchall()

[('2013-11-13 00:00:00', 25.86),
 ('2012-08-05 00:00:00', 23.86),
 ('2010-02-18 00:00:00', 21.86),
 ('2011-04-28 00:00:00', 21.86),
 ('2010-01-18 00:00:00', 18.86)]

Consulta de _invoices_ com origem no Canadá.

In [9]:
cursor.execute(
"""
    SELECT BillingCountry, Total
    FROM invoices
    WHERE BillingCountry == "Canada"
    LIMIT 5;
""")

cursor.fetchall()

[('Canada', 8.91),
 ('Canada', 8.91),
 ('Canada', 0.99),
 ('Canada', 1.98),
 ('Canada', 13.86)]

Consulta do valor total das faturas (_invoices_) agrupadas por país e ordenadas por valor:

In [10]:
cursor.execute("""
    SELECT BillingCountry, SUM(Total) AS TotalAmount
    FROM invoices
    GROUP BY BillingCountry
    ORDER BY TotalAmount DESC
    LIMIT 5;
""")

cursor.fetchall()

[('USA', 523.0600000000003),
 ('Canada', 303.9599999999999),
 ('France', 195.09999999999994),
 ('Brazil', 190.09999999999997),
 ('Germany', 156.48)]

O resultado seguinte responde à questão sobre quais faixas foram compradas por quais países. Para tanto, foram combinadas as tabelas _tracks_, que lista as músicas por nome, _invoices_, que contém os dados de nomes de países e *invoice_items* que contém dados das quantidades de faixas vendidas.

In [11]:
query = """
    SELECT 
        tracks.Name,
        COUNT(invoice_items.TrackId),
        invoices.BillingCountry
    FROM invoices
    JOIN invoice_items
    ON invoices.InvoiceId = invoice_items.InvoiceId
    JOIN tracks
    ON tracks.TrackId = invoice_items.TrackId
    GROUP BY invoices.BillingCountry
    ORDER BY COUNT(invoice_items.TrackId) DESC
    LIMIT 5;
"""

cursor.execute(query)
cursor.fetchall()

[('Your Time Has Come', 494, 'USA'),
 ('Right Through You', 304, 'Canada'),
 ('Morena De Angola', 190, 'France'),
 ('Admirável Gado Novo', 190, 'Brazil'),
 ('Balls to the Wall', 152, 'Germany')]

Término da análise com fechamento da conexão com o banco de dados.

In [12]:
connection.close()